In [1]:
import pandas as pd
import numpy as np

In [2]:
cols = (list(range(-1, 784)))
def prepare_ds(path, split=False, ratio=0.0):
    data = pd.read_csv(filepath_or_buffer=path, names=cols)
    data = data.sample(frac=1)
    if split == True:    
        d1X = np.array(data[:int(-data.shape[0] * ratio)][cols[1:]]).T
        d1Y = np.array(pd.get_dummies(data[:int(-data.shape[0] * ratio)][-1])[:]).T
        d2X = np.array(data[int(-data.shape[0] * ratio):][cols[1:]]).T
        d2Y = np.array(pd.get_dummies(data[int(-data.shape[0] * ratio):][-1])[:]).T
        return d1X, d1Y, d2X, d2Y
    if split == False:
        d1X = np.array(data[:][cols[1:]]).T
        d1Y = np.array(pd.get_dummies(data[:][-1])[:]).T
        return d1X, d1Y

In [3]:
data_path = '../digit-recognizer/mnist_train.csv'
test_path = '../digit-recognizer/mnist_test.csv'
train_X, train_Y, valid_X, valid_Y = prepare_ds(data_path, split=True, ratio=0.2)
test_X, test_Y, = prepare_ds(test_path)

In [4]:
m_train = train_X.shape[1]
m_test = valid_X.shape[1]
train_X.shape, train_Y.shape, valid_X.shape, valid_Y.shape

((784, 48000), (10, 48000), (784, 12000), (10, 12000))

In [18]:
def normalize(X, Y):
    X = (X - np.mean(X, axis=0)) / (np.std(X, axis=0))
    return X, Y

In [13]:
def compute_LogLoss(Yp, Yg):
    cost = -(Yg * np.log(Yp) + (1 - Yg) * np.log(1 - Yp))
    return np.mean(cost)

In [34]:
def accuracy(y_pred, y_actu):
    y_actt = np.argmax(y_actu, axis=0)
    return np.mean((y_pred == y_actt))

In [35]:
def forward(W, b, X, eval=False):
    Z = np.dot(W.T, X) + b
    A = 1. / (1 + np.exp(-Z))
    if eval == False:
        return A
    if eval == True:
        return np.argmax(A, axis=0)

In [58]:
def backward(Yp, Yg, W, b, X, learning_rate=0.1, lambd=0.00):
    cost = compute_LogLoss(Yp.T, Yg.T)
    diff = (Yp - Yg)
    dW = (np.dot(X, diff.T) + (lambd * np.sign(W))) / m_train
    db = np.sum(diff, axis=1, keepdims=True) / m_train
    
    W -= dW * learning_rate
    b -= db * learning_rate
    return cost, W, b

In [59]:
W = np.zeros((784, 10))
b = np.zeros((10, 1))
mini_batch = 6000
new_X, new_Y = normalize(train_X, train_Y) 
for i in range(1, 101):
    for end in range(mini_batch, 48001, mini_batch):
        start = end - mini_batch
        prep_X, prep_Y = new_X[:, start:end], new_Y[:, start:end]
        pred_Y = forward(W, b, prep_X)
        cost, W, b = backward(pred_Y, prep_Y, W, b, prep_X)
    if i % 10 == 0:
        tr_X, tr_Y = prep_X, prep_Y
        vl_X, vl_Y = normalize(valid_X, valid_Y)
        tr_P = forward(W, b, tr_X, eval=True)
        vl_P = forward(W, b, vl_X, eval=True)

        print(i, cost, accuracy(tr_P, tr_Y), accuracy(vl_P, vl_Y))

10 0.13130596607543302 0.8463333333333334 0.8493333333333334
20 0.1113977617239703 0.8628333333333333 0.8665833333333334
30 0.10289924572513716 0.8705 0.8735
40 0.09797225521287646 0.8761666666666666 0.88
50 0.09467297133573485 0.8781666666666667 0.8836666666666667
60 0.09226706831965514 0.882 0.8853333333333333
70 0.09041098300482242 0.8836666666666667 0.8861666666666667
80 0.08892083233345686 0.8858333333333334 0.8880833333333333
90 0.08768851544915893 0.8871666666666667 0.88925
100 0.0866459009549092 0.8878333333333334 0.8903333333333333


1
0.08714329521904246 0.8876666666666667 0.8900833333333333

0.1
0.08669362165359246 0.8878333333333334 0.89025

0.01
0.08665065315585681 0.8878333333333334 0.8903333333333333

0
0.0866459009549092 0.8878333333333334 0.8903333333333333

In [47]:
val_prep_X, val_prep_Y = normalize(valid_X, valid_Y)
valid_preds = forward(W, b, val_prep_X, eval=True)
accuracy(valid_preds, val_prep_Y)

0.9066666666666666

In [48]:
test_prep_X, test_prep_Y = normalize(test_X, test_Y)
test_preds = forward(W, b, test_prep_X, eval=True)
accuracy(test_preds, test_prep_Y)

0.9158

In [3]:
print(23543456543345654323456543345*12345609876567898765678978909876543)

290658329630075232008129876493788552328099955711944879478256335
